In [1]:
import requests
import re
from elasticsearch import Elasticsearch
from elasticsearch.helpers import parallel_bulk
from json import JSONDecodeError
from collections import defaultdict
import json
import numpy as np

from util.es import ES
from smart_dataset.evaluation.dbpedia.evaluate import load_type_hierarchy, evaluate, get_type_path

In [2]:
es = ES(model='EC', similarity='BM25')
if not es.es.indices.exists(es.get_index()):
    # You can allso call this if you want to force reindex or
    # change the documents that are indexed
    es.reindex() 

{'name': 'BIEBER', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'nN32_N-8T1OLuu3AJOSr2g', 'version': {'number': '7.9.1', 'build_flavor': 'default', 'build_type': 'zip', 'build_hash': '083627f112ba94dffc1232e8b42b73492789ef91', 'build_date': '2020-09-01T21:22:21.964974Z', 'build_snapshot': False, 'lucene_version': '8.6.2', 'minimum_wire_compatibility_version': '6.8.0', 'minimum_index_compatibility_version': '6.0.0-beta1'}, 'tagline': 'You Know, for Search'}


In [3]:
# this might report the wrong number if run right after reindexing
print('Num indexed documents: ')
int(es.es.cat.count(es.get_index(), params={"format": "json"})[0]['count'])

Num indexed documents: 


3141300

In [4]:
type_hierarchy, max_depth = load_type_hierarchy('./smart_dataset/evaluation/dbpedia/dbpedia_types.tsv')
with open('./data/train_set_fixed.json', 'r') as f:
    train_set = json.load(f)

Loading type hierarchy from ./smart_dataset/evaluation/dbpedia/dbpedia_types.tsv... 760 types loaded (max depth: 7)


In [5]:
results = es.generate_baseline_scores('train')

In [6]:
k = 100
ground_truth, system_output = {}, {}
for query in train_set:
    ID = query['id']
    if query['category'] != 'resource':
        continue

    ground_truth_type = [t for t in query['type'] if t in type_hierarchy]
    if not ground_truth_type:
        continue
        
    system_output_type = [t for t,s in results[ID] if t in type_hierarchy] if ID in results else None
    system_output_type = get_type_path(system_output_type[0], type_hierarchy) if system_output_type else []
            
    system_output[ID] = {
        'category': 'resource',
        'type': system_output_type
    }
    
    ground_truth[ID] = {
        'category': 'resource',
        'type': ground_truth_type
    }

In [7]:
evaluate(system_output, ground_truth, type_hierarchy, max_depth)



Evaluation results:
-------------------
Category prediction (based on 7662 questions)
  Accuracy: 1.000
Type ranking (based on 7662 questions)
  NDCG@5:  0.247
  NDCG@10: 0.208
